## Chain of Thought & ReAct

- Credits: https://github.com/GoogleCloudPlatform/generative-ai/blob/main/language/prompts/examples/chain_of_thought_react.ipynb

This notebook demonstrates advanced prompting techniques such as chain of thought reasoning, and building ReAct agents using LangChain and Vertex AI. You will start with exploring chain of thought, and how it can be used to improve the performance of language models. Then you will learn how to build ReAct agents using LangChain.

https://cloud.google.com/vertex-ai/docs/generative-ai/learn/overview?hl=es-419

#### Task 1. Setup

In [1]:
# !pip install langchain==0.0.310 \
#               google-cloud-aiplatform==1.35.0 \
#               prettyprinter==0.18.0 \
#               wikipedia==1.4.0 \
#               chromadb==0.3.26 \
#               tiktoken==0.5.1 \
#               tabulate==0.9.0 \
#               sqlalchemy-bigquery==1.8.0 \
#               google-cloud-bigquery==3.11.4

In [2]:
# # Restart kernel after installs so that your environment can access the new packages
# import IPython
# import time

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

#### Task 2. Authentication and import packages

In [3]:
import sys

# Additional authentication is required for Google Colab
if "google.colab" in sys.modules:
    # Authenticate user to Google Cloud
    from google.colab import auth

    auth.authenticate_user()

In [4]:
import sys

# @param {type:"string"}
PROJECT_ID = "vertex-ai-407803"  # Sustituye con tu Project ID
# @param {type:"string"}
LOCATION = "us-central1"  # Elige tu región

if "google.colab" in sys.modules:
    import vertexai
    vertexai.init(project=PROJECT_ID, location=LOCATION)

# import os
# print(os.environ)

import vertexai

from google.oauth2 import service_account
CREDENTIALS = service_account.Credentials.from_service_account_file(r"C:\Users\Usuario\Downloads\google-startup-school\vertex-ai-407803-197b2b2491f7.json")

vertexai.init(project=PROJECT_ID, credentials=CREDENTIALS)

In [5]:
import vertexai
import os
import IPython
from langchain.llms import VertexAI
from IPython.display import display, Markdown

In [6]:
MODEL_NAME = "text-bison@001"  # @param {type:"string"}
llm = VertexAI(model_name=MODEL_NAME, max_output_tokens=1000)

In [7]:
llm.predict(
    "Improve this description : In this notebook we'll explore advanced prompting techniques, and building ReAct agents using LangChain and Vertex AI "
)

'This notebook will explore advanced prompting techniques, and building ReAct agents using LangChain and Vertex AI.\n\nWe will start by introducing the concept of prompting, and how it can be used to improve the performance of language models. We will then discuss some of the advanced prompting techniques that are available, and how they can be used to achieve specific goals. Finally, we will show how to build ReAct agents using LangChain and Vertex AI.\n\nReAct agents are a type of language model that is designed to be used in interactive dialogue systems. They are able to generate text, translate languages, write different kinds of creative content, and answer your questions in an informative way.\n\nLangChain is a library that makes it easy to build and train ReAct agents. It provides a number of features that make it easy to develop and deploy these agents, including a variety of pre-trained language models, a simple API, and support for a variety of tasks.\n\nVertex AI is a manage

### Chain of Thought - Introduction

The technique introduced in this paper is a novel approach to enhance the reasoning capabilities of large language models (LLMs), especially in multi-step reasoning tasks.

In contrast to the standard prompting, where models are asked to directly produce the final answer, 'Chain of Thought Prompting' encourages LLMs to generate intermediate reasoning steps before providing the final answer to a problem. The advantage of this technique lies in its ability to break down complex problems into manageable, intermediate steps. By doing this, the model-generated 'chain of thought' can mimic an intuitive human thought process when working through multi-step problems.

In [8]:
question = """Q: Roger has 5 tennis balls. He buys 2 more cans of tennis balls.
Each can has 3 tennis balls. How many tennis balls does he have now?
A: The answer is 11.
Q: The cafeteria had 23 apples.
If they used 20 to make lunch and bought 6 more, how many apples do they have?
A:"""

llm.predict(question)

'The answer is 19.'

In [9]:
question = """Q: Roger has 5 tennis balls. He buys 2 more cans of tennis balls.
Each can has 3 tennis balls. How many tennis balls does he have now?
A: Roger started with 5 balls. 2 cans of 3 tennis balls
each is 6 tennis balls. 5 + 6 = 11. The answer is 11.
Q: The cafeteria had 23 apples.
If they used 20 to make lunch and bought 6 more, how many apples do they have?
A:"""

llm.predict(question)

'The cafeteria started with 23 apples. They used 20 apples to make lunch, so they have 23 - 20 = 3 apples left. They bought 6 more apples, so they now have 3 + 6 = 9 apples. The answer is 9.'

### Chain of Thought - Zero Shot

Zero-shot CoT prompting is a technique that allows large language models (LLMs) to generate more accurate answers to questions. It does this by appending the words "Let's think step by step." to the end of a question. This simple prompt helps the LLM to generate a chain of thought that answers the question, from which the LLM can then extract a more accurate answer.


In [10]:
question = """Q: Roger has 5 tennis balls. He buys 2 more cans of tennis balls.
Each can has 3 tennis balls. How many tennis balls does he have now?
A: The answer is 11.
Q: The cafeteria had 23 apples.
If they used 20 to make lunch and bought 6 more, how many apples do they have?
A:"""

llm.predict(question)

'The answer is 19.'

In [11]:
question = """Q: Roger has 5 tennis balls. He buys 2 more cans of tennis balls.
Each can has 3 tennis balls. How many tennis balls does he have now?
A: The answer is 11.

Q: The cafeteria had 23 apples.
If they used 20 to make lunch and bought 6 more, how many apples do they have?
A: Let's think step by step."""

llm.predict(question)

'They used 20 apples so they have 23 - 20 = 3 apples left. They bought 6 more apples so they have 3 + 6 = 9 apples.\nThe final answer: 9.'

### Chain of Thought - Self Consistency

An improvement upon CoT prompting is by doing CoT with self-consistency, whereby you generate multiple candidate answers through CoT with the same input.



In [12]:
from operator import itemgetter
from langchain.prompts import PromptTemplate
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough

question = """The cafeteria had 23 apples.
If they used 20 to make lunch and bought 6 more, how many apples do they have?"""

context = """Answer questions showing the full math and reasoning.
Follow the pattern in the example.
"""

one_shot_exemplar = """Example Q: Roger has 5 tennis balls. He buys 2 more cans of tennis balls.
Each can has 3 tennis balls. How many tennis balls does he have now?
A: Roger started with 5 balls. 2 cans of 3 tennis balls
each is 6 tennis balls. 5 + 6 = 11.
The answer is 11.

Q: """


planner = (
    PromptTemplate.from_template(context + one_shot_exemplar + " {input}")
    | VertexAI()
    | StrOutputParser()
    | {"base_response": RunnablePassthrough()}
)

answer_1 = (
    PromptTemplate.from_template("{base_response} A: 33")
    | VertexAI(temperature=0, max_output_tokens=400)
    | StrOutputParser()
)

answer_2 = (
    PromptTemplate.from_template("{base_response} A:")
    | VertexAI(temperature=0.1, max_output_tokens=400)
    | StrOutputParser()
)

answer_3 = (
    PromptTemplate.from_template("{base_response} A:")
    | VertexAI(temperature=0.7, max_output_tokens=400)
    | StrOutputParser()
)

final_responder = (
    PromptTemplate.from_template(
        "Output all the final results in this markdown format: Result 1: {results_1} \n Result 2:{results_2} \n Result 3: {results_3}"
    )
    | VertexAI(max_output_tokens=1024)
    | StrOutputParser()
)

chain = (
    planner
    | {
        "results_1": answer_1,
        "results_2": answer_2,
        "results_3": answer_3,
        "original_response": itemgetter("base_response"),
    }
    | final_responder
)


answers = chain.invoke({"input": question})
display(Markdown(answers))

 Result 1: The question and answer are:

Question: If 23 apples are in the cafeteria, 20 are used for lunch, 6 more are bought, how many apples are there?

Answer: 9

Explanation:

The cafeteria started with 23 apples. They used 20 apples for lunch, so they had 23 - 20 = 3 apples left. They then bought 6 more apples, so they now have 3 + 6 = 9 apples.

The answer should be 9.

Result 2: The cafeteria started with 23 apples. They used 20 apples for lunch, so they had 23 - 20 = 3 apples left. They then bought 6 more apples, so they now have 3 + 6 = 9 apples.
The answer is 9.

Result 3: The cafeteria started with 23 apples. They used 20 apples for lunch, so they had 23 - 20 = 3 apples left. They then bought 6 more apples, so they now have 3 + 6 = 9 apples.
The answer is 9.

### Chain of Thought - JSON Data

In [13]:
context = """Given a JSON entry of a data source, output a JSON with the following fields and explain the reasoning:
pii: True/False, the dataset contains Personally Identifiable Information.
age: How many years since the dataset was last modified.
keywords: New keywords to index this dataset under, beyond the current set of keywords.
The last text output should be the JSON.
"""


question = """
{
    "@type" : "dcat:Dataset",
    "description" : "<p>The MDS 3.0 Frequency Report summarizes information for active residents currently in nursing homes. The source of these counts is the residents MDS assessment record. The MDS assessment information for each active nursing home resident is consolidated to create a profile of the most recent standard information for the resident.</p>\n",
    "title" : "MDS 3.0 Frequency Report",
    "accessLevel" : "public",
    "identifier" : "465",
    "license" : "http://opendefinition.org/licenses/odc-odbl/",
    "modified" : "2016-04-05",
    "temporal" : "2012-01-01T00:00:00-05:00/2015-12-31T00:00:00-05:00",
    "contactPoint" : {
      "@type" : "vcard:Contact",
      "fn" : "Health Data Initiative",
      "hasEmail" : "mailto:HealthData@hhs.gov"
    },
    "bureauCode" : [ "009:38" ],
    "keyword" : [ "Activities of Daily Living (ADL)" ],
    "language" : [ "en" ],
    "programCode" : [ "009:000" ],
    "publisher" : {
      "@type" : "org:Organization",
      "name" : "Centers for Medicare & Medicaid Services",
      "subOrganizationOf" : {
        "@type" : "org:Organization",
        "name" : "Department of Health & Human Services"
      }
    }
  }


"""

llm_prompt = f"{context}\nJSON:{question}\nAnswer:"

display(Markdown(llm.predict(llm_prompt)))

{
  "pii": False,
  "age": 0,
  "keywords": []
}

The dataset does not contain any personally identifiable information. It was last modified in 2016. There are no new keywords to index this dataset under.

In [14]:
one_shot_example = """
JSON:
{

    "@type" : "dcat:Dataset",
    "description" : "The primary purpose of this system of records is to properly pay medical insurance benefits to or on behalf of entitled beneficiaries.",
    "title" : "Medicare Multi-Carrier Claims System",
    "accessLevel" : "restricted public",
    "dataQuality" : true,
    "identifier" : "b6ffafab-1cfd-42dd-b8cb-7a554efaefa7",
    "landingPage" : "http://www.cms.gov/Research-Statistics-Data-and-Systems/Computer-Data-and-Systems/Privacy/Systems-of-Records-Items/09-70-0501-MCS.html",
    "license" : "http://www.usa.gov/publicdomain/label/1.0/",
    "modified" : "2014-09-30",
    "rights" : "Contains personally identifiable information and is subject to the Privacy Act of 1974, as amended at 5 United States Code (U.S.C.) 552a.  Requests should be directed to the appropriate System Manager, identified in the System of Records notice.",
    "primaryITInvestmentUII" : "009-000004256, 009-000004254",
    "systemOfRecords" : "09-70-0501",

    "contactPoint" : {
      "@type" : "vcard:Contact",
      "fn" : "Health Data Initiative",
      "hasEmail" : "mailto:Healthdata@hhs.gov"
    },
    "bureauCode" : [ "009:38" ],
    "keyword" : [ "medicare", "part b", "claims" ],
    "programCode" : [ "009:078" ],
    "theme" : [ "Medicare" ],
    "publisher" : {
      "@type" : "org:Organization",
      "name" : "Centers for Medicare & Medicaid Services",
      "subOrganizationOf" : {
        "@type" : "org:Organization",
        "name" : "Department of Health & Human Services"
      }
    }
  }

Answer: The 'rights' tag says 'Contains personally identifiable information' so pii is True.
The 'modified' tag is '2014-09-30'. The current year is 2023, 2023 minus 2014 is 9, so the age is 9.
To determine keywords I will look at all the fields that describe the dataset.
Then I will take the most salient and distinctive aspects of the fields and make those keywords.
Looking at all the fields, the ones that describe the dataset are  "description" and "title".
The "title" field is "Medicare Multi-Carrier Claims System".
Good keywords from the "title" field are "medicare" and "claims".
The "description" field is ""The primary purpose of this system of records is to properly pay medical insurance benefits to or on behalf of entitled beneficiaries."
Good keywords from the "description" field are "medical insurance benefits".
Good proposed keywords from both fields are "medicare", "claims", and "medical insurance benefits".
Next inspect the "keyword" field to make sure the proposed keywords are not already included.
The "keyword" field contains the keywords "medicare", "part b", and "claims".
From our proposed keywords, "medicare" should not be output since it is already in the "keyword" field.
That leaves "claims" and "medical insurance benefits" as proposed keywords.

Output JSON:
{
  "pii" : true,
  "age" : 9,
  "keywords" : ["claims", "medical insurance benefits"]
}
"""

# Prepending the one shot exemplar before the question we want answered.
llm_prompt = f"{context}{one_shot_example}\nJSON:{question}\nAnswer:"

display(Markdown(llm.predict(llm_prompt)))

The 'accessLevel' tag says 'public' so pii is False.
The 'modified' tag is '2016-04-05'. The current year is 2023, 2023 minus 2016 is 7, so the age is 7.
To determine keywords I will look at all the fields that describe the dataset.
Then I will take the most salient and distinctive aspects of the fields and make those keywords.
Looking at all the fields, the ones that describe the dataset are  "description" and "title".
The "title" field is "MDS 3.0 Frequency Report".
Good keywords from the "title" field are "MDS 3.0" and "frequency report".
The "description" field is "<p>The MDS 3.0 Frequency Report summarizes information for active residents currently in nursing homes. The source of these counts is the residents MDS assessment record. The MDS assessment information for each active nursing home resident is consolidated to create a profile of the most recent standard information for the resident.</p>
".
Good keywords from the "description" field are "nursing home" and "MDS assessment".
Good proposed keywords from both fields are "MDS 3.0", "frequency report", "nursing home", and "MDS assessment".
Next inspect the "keyword" field to make sure the proposed keywords are not already included.
The "keyword" field contains the keyword "Activities of Daily Living (ADL)".
From our proposed keywords, "Activities of Daily Living (ADL)" should not be output since it is already in the "keyword" field.
That leaves "MDS 3.0", "frequency report", "nursing home", and "MDS assessment" as proposed keywords.

Output JSON:
{
  "pii" : false,
  "age" : 7,
  "keywords" : ["MDS 3.0", "frequency report", "nursing home", "MDS assessment"]
}

### ReAct - Introduction

ReAct (short for Reasoning & Acting) combines chain of thought and tool usage together to reason through complex tasks by interacting with external systems. ReAct is particularly useful if you want the LLM or an LLM-based chatbot to reason and take action on external systems through extensions.

- Check function_calling.py: https://github.com/albertgilopez/prompt-engineering/blob/main/function_calling.py
- Check LangChain Agents: https://github.com/albertgilopez/prompt-engineering/blob/main/langchain/langchain_agents.py

For example, LLMs do not know today's date:

In [15]:
llm("What is today's date?")


'Today is Tuesday, March 15, 2023.'

But you can easily create a Python function to fetch today's date:

In [16]:
def get_current_date():
    """
    Gets the current date (today), in the format YYYY-MM-DD
    """

    from datetime import datetime

    todays_date = datetime.today().strftime("%Y-%m-%d")

    return todays_date

To enable the LLM to use this function, you can use tools with a ReAct agent:



In [17]:
from langchain.tools import StructuredTool
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.llms import VertexAI
from langchain.tools import WikipediaQueryRun
from langchain.utilities import WikipediaAPIWrapper
import wikipedia
import vertexai

t_get_current_date = StructuredTool.from_function(get_current_date)

tools = [
    t_get_current_date,
]

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

In [18]:
agent.run("What's today's date?")



> Entering new AgentExecutor chain...
Action:
```
{
  "action": "get_current_date",
  "action_input": {}
}
```

Observation: 2024-01-24
Thought:I know what to respond
Action:
```
{
  "action": "Final Answer",
  "action_input": "Today is 2024-01-24"
}
```

> Finished chain.


'Today is 2024-01-24'

### ReAct - Wikipedia
In the example below, you can enable the LLM the check Wikipedia:

In [19]:
llm = VertexAI(temperature=0)

_ = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

tools = load_tools(["wikipedia"], llm=llm)

tools.append(t_get_current_date)

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

# agent.run("What former US President was first?")
agent.run(
    "Fetch today's date, and tell me which famous person was born or who died on the same day as today?"
)



> Entering new AgentExecutor chain...
 Action:
```
{
  "action": "get_current_date",
  "action_input": {}
}
```

Observation: 2024-01-24
Thought: Action:
```
{
  "action": "Wikipedia",
  "action_input": {
    "query": "January 24"
  }
}
```

Observation: Page: January 24
Summary: January 24 is the 24th day of the year in the Gregorian calendar;  341 days remain until the end of the year (342 in leap years).



Page: Tornadoes of 2023
Summary: This page documents notable tornadoes and tornado outbreaks worldwide in 2023. Strong and destructive tornadoes form most frequently in the United States, Argentina, Brazil, Bangladesh, and Eastern India, but can occur almost anywhere under the right conditions. Tornadoes also develop occasionally in southern Canada during the Northern Hemisphere's summer and somewhat regularly at other times of the year across Europe, Asia, Argentina, Australia and New Zealand. Tornadic events are often accompanied by other forms of severe weather, including th

'Today is January 24, 2024. Notable events on this day include the birth of the composer Wolfgang Amadeus Mozart in 1756 and the death of the artist Salvador Dalí in 1989.'

### ReAct - BigQuery

In [20]:
import re
from typing import Sequence, List, Tuple, Optional, Any
from langchain.agents.agent import Agent, AgentOutputParser
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.base import BasePromptTemplate
from langchain.tools.base import BaseTool
from langchain.agents import Tool, initialize_agent, AgentExecutor
from langchain.llms import VertexAI
from langchain.agents.react.output_parser import ReActOutputParser
import pandas as pd
from google.cloud import bigquery
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.tools.python.tool import PythonREPLTool

bq = bigquery.Client(project=PROJECT_ID, credentials=CREDENTIALS)
llm = VertexAI(temperature=0, max_tokens=1024)

Define custom tools:

In [21]:
def get_comment_by_id(id: str) -> str:
    QUERY = "SELECT text FROM bigquery-public-data.hacker_news.full WHERE ID = {id} LIMIT 1".format(
        id=id
    )
    df = bq.query(QUERY).to_dataframe()

    return df


def get_comment_by_user(user):
    QUERY = "SELECT text FROM bigquery-public-data.hacker_news.full WHERE `BY` = {user} LIMIT 10".format(
        user=user
    )
    df = bq.query(QUERY).to_dataframe()

    return df


def generate_response_for_comment(comment):
    question = """Create a 1 sentence friendly response to the following comment: {comment}""".format(
        comment=comment
    )
    llm1 = VertexAI(temperature=0.3, max_output_tokens=150)
    response = llm1.predict(question)

    return response


def generate_sentiment_for_comment(comment):
    question = """What is the sentiment of the comment (Negative, Positive, Neutral): {comment}""".format(
        comment=comment
    )
    llm1 = VertexAI(temperature=0.3, max_output_tokens=150)
    response = llm1.predict(question)

    return response


def generate_category_for_comment(comment):
    question = """Put the comment into one of these categories (Technology, Politics, Products, News): {comment}""".format(
        comment=comment
    )
    llm1 = VertexAI(temperature=0.3, max_output_tokens=150)
    response = llm1.predict(question)

    return response

In [22]:
tools = [
    Tool(
        name="GetCommentsById",
        func=get_comment_by_id,
        description="Get a pandas dataframe of comment by id.",
    ),
    Tool(
        name="GetCommentsByUser",
        func=get_comment_by_user,
        description="Get a pandas dataframe of comments by user.",
    ),
    Tool(
        name="GenerateCommentResponse",
        func=generate_response_for_comment,
        description="Get an AI response for the user comment.",
    ),
    Tool(
        name="GenerateCommentSentiment",
        func=generate_sentiment_for_comment,
        description="Get an AI sentiment for the user comment.",
    ),
    Tool(
        name="GenerateCategorySentiment",
        func=generate_category_for_comment,
        description="Get an AI category for the user comment.",
    ),
    PythonREPLTool(),
]

Setup Prompt and Examples:

In [23]:
EXAMPLES = [
    """Question: Write a response to the following Comment 1234 ?
Thought: I need to get comment 1234 using GetCommentsById.
Action: GetCommentsById[1234]
Observation: "Comment Text"
Thought: I need to generate a response to the comment.
Action: GenerateCommentResponse["Comment Text"]
Observation: LLM Generated response
Thought: So the answer is "LLM Generated response".
Action: Finish["LLM Generated response"],
Question: Write a response to all the comments by user xx234 ?
Thought: I need to get all the comments by xx234 using GetCommentsByUser.
Action: GetCommentsByUser['xx234']
Observation: "Comment Text"
Thought: I need to generate a response to each comment.
Action: GenerateCommentResponse["Comment Text 1"]
Observation: "LLM Generated response 1"
Thought: I need to generate a response to each comment.
Action: GenerateCommentResponse["Comment Text 2"]
Observation: "LLM Generated response 2"
Thought: I need to generate a response to each comment.
Action: GenerateCommentResponse["Comment Text 3"]
Observation: "LLM Generated response 3"
Thought: I Generated responses for all the comments.
Action: Finish["Done"],
Question: Sentiment for all the comments by user xx234 ?
Thought: I need to get all the comments by xx234 using GetCommentsByUser.
Action: GetCommentsByUser['xx234']
Observation: "Comment Text"
Thought: I need to determine sentiment of each comment.
Action: GenerateCommentSentiment["Comment Text 1"]
Observation: "LLM Generated Sentiment 1"
Thought: I need to determine sentiment of each comment.
Action: GenerateCommentSentiment["Comment Text 2"]
Observation: "LLM Generated Sentiment 2"
Thought: I need to generate a response to each comment.
Action: GenerateCommentSentiment["Comment Text 3"]
Observation: "LLM Generated Sentiment 3"
Thought: I determined sentiment for all the comments.
Action: Finish["Done"],
Question: Category for all the comments by user xx234 ?
Thought: I need to get all the comments by xx234 using GetCommentsByUser.
Action: GetCommentsByUser['xx234']
Observation: "Comment Text"
Thought: I need to determine the category of each comment.
Action: GenerateCategorySentiment["Comment Text 1"]
Observation: "LLM Generated Category 1"
Thought: I need to determine category of each comment.
Action: GenerateCategorySentiment["Comment Text 2"]
Observation: "LLM Generated Category 2"
Thought: I need to generate a category to each comment.
Action: GenerateCategorySentiment["Comment Text 3"]
Observation: "LLM Generated Category 3"
Thought: I determined Category for all the comments.
Action: Finish["Done"]
"""
]

SUFFIX = """\nIn each action, you cannot use the nested functions, such as GenerateCommentResponse[GetCommentsByUser["A"], GetCommentsById["B"]].
Instead, you should parse into 3 actions - GetCommentsById['A'], GetCommentsByUser['B'], and GenerateCommentResponse("Comment").

Let's start.

Question: {input}
{agent_scratchpad} """

output_parser = CommaSeparatedListOutputParser()

format_instructions = output_parser.get_format_instructions()

TEST_PROMPT = PromptTemplate.from_examples(
    examples=EXAMPLES,
    suffix=SUFFIX,
    input_variables=["input", "agent_scratchpad"],
)


class ReActTestAgent(Agent):
    @classmethod
    def _get_default_output_parser(cls, **kwargs: Any) -> AgentOutputParser:
        return ReActOutputParser()

    @classmethod
    def create_prompt(cls, tools: Sequence[BaseTool]) -> BasePromptTemplate:
        return TEST_PROMPT

    @classmethod
    def _validate_tools(cls, tools: Sequence[BaseTool]) -> None:
        if len(tools) != 6:
            raise ValueError("The number of tools is invalid.")
        tool_names = {tool.name for tool in tools}
        if tool_names != {
            "GetCommentsById",
            "GetCommentsByUser",
            "GenerateCommentResponse",
            "GenerateCommentSentiment",
            "GenerateCategorySentiment",
            "Python_REPL",
        }:
            raise ValueError("The name of tools is invalid.")

    @property
    def _agent_type(self) -> str:
        return "react-test"

    @property
    def finish_tool_name(self) -> str:
        return "Final Answer: "

    @property
    def observation_prefix(self) -> str:
        return f"Observation: "

    @property
    def llm_prefix(self) -> str:
        return f"Thought: "

In [24]:
llm = VertexAI(
    temperature=0,
)

agent = ReActTestAgent.from_llm_and_tools(llm, tools, verbose=True)

agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True
)
agent_executor.handle_parsing_errors = True

In [25]:
# !pip install db-dtypes

In [ ]:
input = "Get the category for comment 8885404"
agent_executor.run(input)

In [ ]:
input = "Get the sentiment for comment 8885404"
agent_executor.run(input)

In [ ]:
input = "Get a response for comment 8885404"
agent_executor.run(input)

In [29]:
input = "Get the sentiment for all the to comments written by chris"
agent_executor.run(input)



> Entering new AgentExecutor chain...
 Thought: I need to get all the comments by chris using GetCommentsByUser.
Action: GetCommentsByUser['chris']
Observation:                                                 text
0  We (Weebly) had 18Gbps of UDP&#x2F;123 (NTP) t...
1                                 excellent article.
2  SSL requires additional CPU resources, and is ...
3                                               None
4                                               None
5  More specifically, it broke down at metadata o...
6  I agree.  Simply disabling an account that was...
7   Back in 2014 they wanted a $10k&#x2F;yr minimum.
8  This happened to us (Weebly) years ago when we...
9                                               None
Thought:  I need to determine the sentiment of each comment.
Action: GenerateCommentSentiment["We (Weebly) had 18Gbps of UDP\u2f123 (NTP) t..."]
Observation:  Neutral
Thought:  I need to determine the sentiment of each comment.
Action: GenerateCommentSen

'"Done"'

In [30]:
input = "Get the response for all the to comments written by chris."
agent_executor.run(input)



> Entering new AgentExecutor chain...
Could not parse LLM Output:  Thought: I need to get all the comments by chris using GetCommentsByUser.
  Action: GetCommentsByUser['chris']
  Observation: "Comment Text"
  Thought: I need to generate a response to each comment.
  Action: GenerateCommentResponse["Comment Text"]
  Observation: "LLM Generated response"
  Thought: So the answer is "LLM Generated response".
  Action: Finish["LLM Generated response"],
Observation: Invalid or incomplete response
Thought:  I need to get all the comments by chris using GetCommentsByUser.
Action: GetCommentsByUser['chris']
Observation:                                                 text
0  We (Weebly) had 18Gbps of UDP&#x2F;123 (NTP) t...
1                                 excellent article.
2  SSL requires additional CPU resources, and is ...
3                                               None
4                                               None
5  More specifically, it broke down at metadata o...
6  I a